In [1]:
#https://petamind.com/movielens-automation-process-and-export/
#https://petamind.com/build-a-simple-recommender-system-with-matrix-factorization/
#https://petamind.com/movielens-automation-process-and-export/
import math
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import keras
from keras.layers import Input, LSTM
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.models import Model
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

from sklearn import metrics
class DataGenerator(Sequence):
    def __init__(self, dataset, vect, le, show_debug = True, batch_size=4, dim=(1), shuffle=False):
        'Initialization'
        self.dim = dim
        self.show_debug = show_debug
        self.batch_size = batch_size
        self.dataset = dataset
        self.vect = vect
        self.le = le
        self.shuffle = shuffle
        self.indexes = dataset.index
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return math.ceil(len(self.dataset) / self.batch_size)
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        idxs = [i for i in range(index*self.batch_size,(index+1)*self.batch_size)]
    
        # Find list of IDs
        list_IDs_temp = [self.indexes[k] for k in idxs]

        
        # Generate data
#         User = dataset.loc[list_IDs_temp,['user_id']].to_numpy()#.reshape(-1)
#         Item = dataset.loc[list_IDs_temp,['item_id']].to_numpy()#.reshape(-1)
        y = dataset.loc[list_IDs_temp,['tweet_class']]#.reshape(-1)
        X = dataset.loc[list_IDs_temp,['tweet_text']]
#         print("X", X["tweet_text"])
        X = self.vect.transform(X["tweet_text"])
        y = self.le.transform(y)
        if self.show_debug:
            print("self.indexes", self.indexes)
            print("index*self.batch_size, (index+1)*self.batch_size)", index*self.batch_size, (index+1)*self.batch_size)
            print("idxs", idxs)
            print("list_IDs_temp", list_IDs_temp)
            print("X", type(X), X.shape, X.todense())
            print("y", type(y), y.shape)

#         print("u,i,r:", [User, Item], [y])
#         return [User, Item],[y]
        return X.todense(), y
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(dataset))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
# train_generator = DataGenerator(dataset=dataset, vect = vect, le = le, show_debug = True)
# x, y = train_generator.__getitem__(0)
# # print("x:", x,"\ny:", y, "x.shape:", x.shape,"\ny.shape:", y.shape)

Using TensorFlow backend.


In [2]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2
from libraries.feauture_extraction.Feature_Extraction import Word2vec_Rules_Features as fe
from libraries.training_clf.Training_classifier import Training_Classifiers as te

def simple_CountVect(df, lower, higher, minDF= 1.0, maxDF = 1.0):
    Vect = CountVectorizer(ngram_range=(lower, higher))
    Vect.fit(df["tweet_text"])
    return Vect
lower = 1
higher = 1
path = "../datasets/"
filename = "500_random_sample.csv"
batch_size = 64
# filename = "nepal_earthquake.csv"
include_rules = "yes"
path_to_file = path + filename
path_to_nonProcessed = path + filename
dataset = pd.read_csv(path_to_file)
label = ["request", "not_excl"]
le = preprocessing.LabelEncoder()
le = le.fit(label)
nb_classes = len(label)
dataset= dataset.drop(list(range(dataset.shape[0]-(dataset.shape[0]%batch_size),dataset.shape[0])))
vect = simple_CountVect(dataset, lower = lower, higher = higher)

# model = te.apply_LSTM_KF(len(vect.get_feature_names()), 1)
te = te()
# model = te.apply_GRU_KF(len(vect.get_feature_names()), 1)
model = te.apply_LSTM_KF(len(vect.get_feature_names()), 1)
# model = te.apply_RNN_KF(len(vect.get_feature_names()), 1)
# model = te.apply_CNN_KF(len(vect.get_feature_names()), 1)

# model.summary()
# list_IDs_temp = [0, 1, 2]
# X = dataset.loc[list_IDs_temp,['tweet_text']]
# print(X["tweet_text"])
# print((vect.transform(X["tweet_text"])).shape)
# x, y = train_generator.__getitem__(1)
# print("x:", x,"\ny:", y)
# print(le.transform(label))
print(dataset.shape[0]/batch_size)


scritp completed

Total time for script completion :0:00:00

scritp completed

Total time for script completion :0:00:00
7.0


In [3]:
model.summary()
#create an instance of the generator with proper dataset
train_generator = DataGenerator(dataset=dataset, vect = vect, le = le, show_debug = False, batch_size=batch_size)
#train
history = model.fit_generator(train_generator)#, epochs=2)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1196)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1196, 1196)        1430416   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1196, 128)         678400    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1196, 128)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1196, 128)         131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1196, 128)         0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               1315

In [4]:
y_pred = np.argmax(model.predict_generator(train_generator), axis=1)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:
y_pred = model.predict_generator(train_generator)
print(y_pred, y_pred.shape)
y_pred = np.argmax(model.predict_generator(train_generator), axis=1)

print(y_pred, y_pred.shape)

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.